In [140]:
import wave
import os
import glob
import soundfile as sf
from scipy.signal import spectrogram, butter, lfilter, decimate, hilbert, resample
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

    
VOLUME_AMP_MULTIPLE = 20
NPERSEG = 256
NOVERLAP = int(NPERSEG * 0.25)
WINDOW = 'hanning'
SPECTROGRAM_RAW_LOW = 1
SPECTROGRAM_RAW_HIGH = 4
SPECTROGRAM_POWER_FACTOR = 4
LOWCUT = 4500
HIGHCUT = 9500
SPECTROGRAM_HEIGHT = int(64)
morph_elem = 0
morph_size = 0
morph_operator = 0
element=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(6,6))

def fade_in(data, samples):
    samples=math.ceil(samples)
    g=np.linspace(0,1,samples)
    data[: samples]=np.multiply(data[:samples],g)
    return data;

def fade_out(data, samples):
    samples=math.ceil(samples)
    g=np.linspace(1,0,samples)
    data[(len(data)-samples) :]=np.multiply(data[(len(data)-samples) :],g)
    return data;

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
    
def getEnvelope(inputSignal,intervalLength):
    absoluteSignal = []
    print('getting envelope..')
    for sample in inputSignal:
        absoluteSignal.append (abs (sample))
    outputSignal = []
    for baseIndex in range (0, len (absoluteSignal)):
        maximum = 0
        for lookbackIndex in range (intervalLength):
            maximum = max (absoluteSignal [baseIndex - lookbackIndex], maximum)
        outputSignal.append (maximum)
    print('done getting envelope..')
    return outputSignal 


def wav_to_spectrogram(data, samplerate): 
    #makes spectrogram, resizes to 100 square, flattens    
    f, t, spec = spectrogram(data,fs=float(samplerate),window=WINDOW,nperseg=NPERSEG,noverlap=NOVERLAP,mode='magnitude')    
    sel=np.where(np.logical_and(f>=3000, f<=12000))
    spec=spec[sel][:]
    spec=np.flipud(spec)
    spec=spec+0.0001
    #spec(np.nonzero(spec))=spec(np.nonzero(spec)))
    spec=np.log10(spec)*20
    spec = cv2.resize(spec, (100, 100))  
    norm_image = np.zeros((100,100))
    cv2.normalize(spec, norm_image, 0, 1, norm_type=cv2.NORM_MINMAX)
    print(np.min(norm_image), np.max(norm_image))
    tmp = norm_image.reshape(1,10000)
    p = np.percentile(tmp, 85)
    th, im_th = cv2.threshold(norm_image, p, 1, cv2.THRESH_TOZERO)
    #im=im_th
    im = im_th.reshape(1,10000)    
    return im;    
    

#define moving average function
def moving_avg(x, n):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[n:] - cumsum[:-n]) / float(n)    
    
    
def load_wavs(directory):
    os.chdir(directory)
    wavefiles = glob.glob('*.wav')
    imstack=pd.DataFrame()
    for idx, wavefile in enumerate(wavefiles):
        data, samplerate = sf.read(wavefile) 
        data=fade_in(data,0.005*samplerate)
        data=fade_out(data,0.005*samplerate)
        target_length=round(samplerate*0.5)
        actual_length=len(data)
        difference=target_length-actual_length
        d=round(difference/2)
    
        if(d>0):
            data=np.pad(data, (d, d), 'constant')
        if(d<0):
            d=d*-1
            data=data[d :]
            q=len(data)-d
            data=data[: q]  
    
        f, t, spec = spectrogram(data,fs=float(samplerate),window=WINDOW,nperseg=NPERSEG,noverlap=NOVERLAP,mode='magnitude')
        spec=np.flipud(spec)
        spec=spec[:45][:]
        spec = cv2.resize(spec, (100, 100))   
        p = np.percentile(spec, 98)
        th, im_th = cv2.threshold(spec, p, 1, cv2.THRESH_BINARY)
        im = im_th.reshape(1,10000)
        if idx==0:
            imstack=im
        if idx>0:
            imstack=np.append(imstack,im,axis=0)   
    return imstack, idx;

def max_rolling1(a, window,axis =1):
        shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
        strides = a.strides + (a.strides[-1],)
        rolling = np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
        return np.max(rolling,axis=axis)

def load_wavs_chop(filename):
    wavefiles = glob.glob('*.wav')
    data, samplerate = sf.read(filename)
    data=data*100
    data=butter_bandpass_filter(data, 4000, 10000, samplerate, order=5)
    
    plt.plot(data[:round(0.5*samplerate)])# plotting the graph
   
    data_ds=decimate(data, 4)
    samplerate_ds=int(samplerate/4)
    data_ds=data
    samplerate_ds=samplerate
    
    analytic_signal = hilbert(data_ds)
    amplitude_envelope = np.abs(analytic_signal)
    #amplitude_envelope = moving_avg(amplitude_envelope,500)
    amplitude_envelope = max_rolling1(amplitude_envelope,200)
    #amplitude_envelope=getEnvelope(data_ds,400)
    #plt.plot(amplitude_envelope)# plotting the graph
    #plt.show() #Displaying the figures
    call=np.zeros(len(amplitude_envelope), dtype=float, order='C')
    #print(len(call))
    envelope_threshold=np.percentile(amplitude_envelope, 20)
    envelope_threshold=0.02
    for i in range(0, len(call)):
        if(amplitude_envelope[i]>envelope_threshold):
            call[i]=1
    #a=resample(call, len(call)*4)
    a=call
    plt.plot(a[:round(0.5*samplerate)])   
    plt.plot(amplitude_envelope[:round(0.5*samplerate)])
    plt.show() #Displaying the figures 
    
    return call, data, data_ds, samplerate_ds, samplerate;        
   
def show_spectrograms(imstack, idx):
    rows=math.ceil(math.sqrt(idx+1))
    cols=rows
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(15, 15))  
    for idx, wavefile in enumerate(wavefiles):
        im=imstack[idx][:].reshape(100,100)
        #ax.ravel()[idx].imshow(im,cmap='binary')
        ax.ravel()[idx].imshow(im,cmap='gray')
        ax.ravel()[idx].set_title(wavefiles[idx])
        ax.ravel()[idx].set_axis_off()
    plt.show()  
    
    
def show_spectrograms_column(imstack, idx):
    rows=idx+1
    cols=1
    fig, ax = plt.subplots(nrows=rows, ncols=cols ) 

    for idx, wavefile in enumerate(wavefiles):
        im=imstack[idx][:].reshape(100,100)
        #ax.ravel()[idx].imshow(im,cmap='binary')
        ax.ravel()[idx].imshow(im)
        ax.ravel()[idx].set_title(wavefiles[idx])
        ax.ravel()[idx].set_axis_off()
    fig.tight_layout()
    plt.show()      

def set_to_target_length(data,target_length):
    q=len(data)-target_length
    #print(data.shape)
    #print(target_length, len(data), q)
    if(q==0):
        data=fade_in(data,0.005*samplerate)
        data=fade_out(data,0.005*samplerate)
    if q<0:
        data=fade_in(data,0.005*samplerate)
        data=fade_out(data,0.005*samplerate)
        data=np.append(data,np.zeros(abs(q)))
        #print('q<0, ',  len(data))
    if q>0:
        data=data[:target_length]
        data=fade_in(data,0.005*samplerate)
        data=fade_out(data,0.005*samplerate) 
        #print('q>0, ', len(data))
    return data;
     


/Users/rogierlandman/from_Samsung/Samvaran_training_data/Long_filtered_Tr.wav
